# 🚀 YOLOv11s 소나무 피해목 탐지 - 스마트 학습 시스템

## 🎯 **3단계 스마트 학습 워크플로**

### **⚡ 빠른 시작**
1. **🖥️ GPU 설정**: `런타임` → `런타임 유형 변경` → `T4 GPU` 선택
2. **🚀 단계별 실행**: 1단계부터 순서대로 실행  
3. **☕ 대기**: 약 1-2시간 후 완성!

### **🔄 3단계 실행 플로우**
- **🔧 1단계**: 환경 설정 및 훈련 데이터 자동 탐지
- **🎓 2단계**: YOLOv11s 멀티스케일 학습 + Google Drive 백업
- **🎯 3단계**: 학습된 모델 성능 테스트 및 탐지 결과 시각화

### **🎯 네이밍 규칙**
- **프로젝트**: `pinetree_multiscale` (멀티스케일 최적화)
- **실행명**: `multiscale_training_YYYYMMDD_HHMM`
- **예시**: `multiscale_training_20250925_1430`

### **📈 목표 성능 (mAP 0.65+ 달성)**
- **mAP50**: 65%+ (멀티스케일로 성능 향상)
- **Precision**: 70%+ (오탐지 최소화)  
- **Recall**: 60%+ (실제 피해목 탐지율)
- **추론 속도**: Tesla T4에서 실시간 처리

---

## 🔧 **1단계: 환경 설정 및 훈련 데이터 자동 탐지**

> YOLOv11 설치, GPU 최적화, 훈련 데이터셋 자동 검색을 수행합니다.

In [ ]:
# 🔧 1단계: 환경 설정 및 기존 모델 탐지
import subprocess
import sys
import os
import glob
from datetime import datetime
from pathlib import Path

def install_package(package):
    """패키지 자동 설치"""
    try:
        __import__(package.split('==')[0])
        print(f"✅ {package} 이미 설치됨")
    except ImportError:
        print(f"📦 {package} 설치 중...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} 설치 완료")

print("🚀 YOLOv11s 소나무 피해목 탐지 - 환경 설정")
print("=" * 50)

# � 필수 패키지 설치 (구글 코랩 환경)
print("📦 필수 패키지 설치 확인:")
required_packages = [
    "ultralytics",
    "torch", 
    "torchvision",
    "opencv-python",
    "pillow",
    "matplotlib",
    "numpy",
    "pandas",
    "pyyaml"
]

for package in required_packages:
    install_package(package)

print("✅ 모든 패키지 설치 확인 완료\n")

# �🗂️ Google Drive 마운트 및 ZIP 데이터 자동 압축 해제 (Colab 환경에서만)
try:
    import google.colab
    IN_COLAB = True
    print("📍 Google Colab 환경 감지")
    
    # Google Drive 마운트
    print("🔗 Google Drive 마운트 중...")
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive 마운트 완료")
    
    # 작업 디렉토리 설정
    drive_path = '/content/drive/MyDrive/pinetree_scan'
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
        print(f"📁 작업 디렉토리 생성: {drive_path}")
    
    # 🔍 ZIP 파일 자동 탐지 및 압축 해제 함수
    def find_and_extract_training_zip():
        """Google Drive에서 training_data ZIP 파일을 찾아 압축 해제"""
        import zipfile
        import shutil
        
        print("\n🔍 Google Drive에서 ZIP 데이터셋 탐지 중...")
        
        # ZIP 파일 검색 경로 (우선순위별)
        zip_search_paths = [
            # 1순위: complete_training_dataset_ 패턴 (최우선 - 최신 학습 데이터셋)
            '/content/drive/MyDrive/pinetree_scan/complete_training_dataset_*.zip',
            '/content/drive/MyDrive/complete_training_dataset_*.zip',
            
            # 2순위: pinetree_scan/training_data 폴더 내의 모든 ZIP 파일
            '/content/drive/MyDrive/pinetree_scan/training_data/*.zip',
            
            # 3순위: pinetree_scan 폴더 내
            '/content/drive/MyDrive/pinetree_scan/training_data.zip',
            '/content/drive/MyDrive/pinetree_scan/dataset.zip', 
            '/content/drive/MyDrive/pinetree_scan/data.zip',
            
            # 4순위: 직접 업로드
            '/content/drive/MyDrive/training_data.zip',
            '/content/drive/MyDrive/dataset.zip',
            '/content/drive/MyDrive/data.zip',
            '/content/drive/MyDrive/yolo_data.zip',
            '/content/drive/MyDrive/pine_data.zip',
            
            # 5순위: 기타 와일드카드 검색
            '/content/drive/MyDrive/*training*.zip',
            '/content/drive/MyDrive/*dataset*.zip',
            '/content/drive/MyDrive/*data*.zip',
            '/content/drive/MyDrive/pinetree_scan/*training*.zip',
            '/content/drive/MyDrive/pinetree_scan/*dataset*.zip'
        ]
        
        found_zips = []
        
        for search_path in zip_search_paths:
            if '*' in search_path:
                # 와일드카드 검색
                import glob
                matches = glob.glob(search_path)
                for match in matches:
                    if os.path.exists(match) and match.endswith('.zip'):
                        found_zips.append(match)
            else:
                # 직접 경로 확인
                if os.path.exists(search_path):
                    found_zips.append(search_path)
        
        if found_zips:
            # 중복 제거
            found_zips = list(set(found_zips))
            
            print(f"📦 발견된 ZIP 파일들:")
            for i, zip_file in enumerate(found_zips, 1):
                file_size = os.path.getsize(zip_file) / 1024 / 1024  # MB
                print(f"  {i}. {os.path.basename(zip_file)} ({file_size:.1f}MB)")
                print(f"     경로: {zip_file}")
            
            # 파일명에서 날짜 추출하여 최신 파일 선택
            def extract_date_from_filename(filename):
                """파일명에서 날짜 패턴(YYYYMMDD_HHMMSS) 추출"""
                import re
                # complete_training_dataset_지역명_20250925_150027.zip 패턴
                match = re.search(r'(\d{8}_\d{6})', filename)
                if match:
                    return match.group(1)
                # 다른 날짜 패턴들
                match = re.search(r'(\d{8})', filename)
                if match:
                    return match.group(1) + "_000000"
                return "19700101_000000"  # 기본값 (가장 오래된 날짜)
            
            # complete_training_dataset_ 패턴이 있는지 확인
            complete_training_zips = [f for f in found_zips if 'complete_training_dataset_' in os.path.basename(f)]
            
            if complete_training_zips:
                # complete_training_dataset_ 파일 중 가장 최신 날짜 선택
                zip_path = max(complete_training_zips, key=lambda x: extract_date_from_filename(os.path.basename(x)))
                print(f"\n🎯 최신 complete_training_dataset 파일 자동 선택!")
                
                # 날짜 및 지역 정보 추출
                filename = os.path.basename(zip_path)
                date_match = extract_date_from_filename(filename)
                if date_match != "19700101_000000":
                    date_str = f"{date_match[:4]}-{date_match[4:6]}-{date_match[6:8]} {date_match[9:11]}:{date_match[11:13]}:{date_match[13:15]}"
                    print(f"  📅 데이터셋 날짜: {date_str}")
                
                # 지역명 추출
                import re
                region_match = re.search(r'complete_training_dataset_([^_]+)_\d{8}_\d{6}\.zip', filename)
                if region_match:
                    region_name = region_match.group(1)
                    print(f"  🌍 대상 지역: {region_name}")
                    
            else:
                # 기본 로직: 가장 큰 파일을 선택 (훈련 데이터가 클 가능성이 높음)
                zip_path = max(found_zips, key=os.path.getsize)
                print(f"\n✅ 크기 기준으로 선택된 파일:")
                
            print(f"📁 최종 선택: {os.path.basename(zip_path)}")
            print(f"📍 전체 경로: {zip_path}")
            
            # 파일 크기 확인
            zip_size = os.path.getsize(zip_path) / 1024 / 1024  # MB
            print(f"📏 크기: {zip_size:.1f}MB")
            
            # 압축 해제 대상 경로
            extract_path = '/content/drive/MyDrive/pinetree_scan/training_data'
            temp_extract_path = '/content/training_data_temp'
            
            try:
                print(f"\n🔄 ZIP 파일 압축 해제 중...")
                
                # 임시 경로에 압축 해제
                if os.path.exists(temp_extract_path):
                    shutil.rmtree(temp_extract_path)
                
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(temp_extract_path)
                
                print(f"✅ 압축 해제 완료: {temp_extract_path}")
                
                # 압축 해제된 내용 확인 및 정리
                extracted_items = os.listdir(temp_extract_path)
                print(f"📋 압축 해제 내용: {extracted_items}")
                
                # training_data 폴더 구조 정리
                final_path = extract_path
                
                # 기존 training_data가 있으면 삭제
                if os.path.exists(final_path):
                    shutil.rmtree(final_path)
                    print(f"🗑️ 기존 데이터 삭제: {final_path}")
                
                # 압축 해제된 내용이 training_data 폴더인지 확인
                if 'training_data' in extracted_items:
                    # training_data 폴더가 포함된 경우
                    source_path = os.path.join(temp_extract_path, 'training_data')
                    shutil.move(source_path, final_path)
                elif len(extracted_items) == 1 and os.path.isdir(os.path.join(temp_extract_path, extracted_items[0])):
                    # 단일 폴더인 경우
                    source_path = os.path.join(temp_extract_path, extracted_items[0])
                    shutil.move(source_path, final_path)
                else:
                    # 파일들이 직접 압축된 경우
                    shutil.move(temp_extract_path, final_path)
                
                # 임시 폴더 정리
                if os.path.exists(temp_extract_path):
                    shutil.rmtree(temp_extract_path)
                
                # 압축 해제 결과 확인
                if os.path.exists(final_path):
                    contents = os.listdir(final_path)
                    print(f"📂 최종 경로: {final_path}")
                    print(f"📋 내용: {contents}")
                    
                    # 기본 구조 확인
                    images_dir = os.path.join(final_path, 'images')
                    labels_dir = os.path.join(final_path, 'labels')
                    
                    if os.path.exists(images_dir):
                        image_count = len([f for f in os.listdir(images_dir) 
                                         if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.tiff'))])
                        print(f"📸 이미지: {image_count}개")
                    
                    if os.path.exists(labels_dir):
                        label_count = len([f for f in os.listdir(labels_dir) 
                                         if f.endswith('.txt')])
                        print(f"🏷️ 라벨: {label_count}개")
                
                return final_path
                
            except Exception as e:
                print(f"❌ ZIP 압축 해제 실패: {e}")
                return None
        
        else:
            print("❌ ZIP 파일을 찾을 수 없습니다.")
            print("💡 다음 경로에 ZIP 파일을 업로드하세요:")
            print("  📦 /content/drive/MyDrive/pinetree_scan/training_data/your_data.zip")
            print("  📦 /content/drive/MyDrive/pinetree_scan/training_data.zip")
            print("  📦 /content/drive/MyDrive/training_data.zip")
            print("  📦 /content/drive/MyDrive/dataset.zip")
            return None
    
    # ZIP 파일 압축 해제 실행
    extracted_data_path = find_and_extract_training_zip()
    
    # 학습 데이터 경로 확인 (압축 해제 결과 포함)
    data_paths = [
        extracted_data_path,  # 압축 해제된 경로 우선
        '/content/drive/MyDrive/pinetree_scan/training_data',
        '/content/training_data',
        '/content/drive/MyDrive/training_data'
    ]
    
    data_path = None
    for path in data_paths:
        if path and os.path.exists(path):
            # data.yaml이 있거나 images 폴더가 있으면 유효한 경로로 간주
            if (os.path.exists(f"{path}/data.yaml") or 
                os.path.exists(f"{path}/images")):
                data_path = path
                print(f"✅ 학습 데이터 발견: {data_path}")
                break
    
    if not data_path:
        print("⚠️ 학습 데이터를 찾을 수 없습니다.")
        print("📋 필요한 파일 구조:")
        print("  📦 ZIP 파일:")
        print("    📂 training_data.zip")
        print("      📂 images/ (이미지 파일들)")
        print("      📂 labels/ (라벨 파일들)")
        print("      📄 data.yaml (선택사항)")
        print("")
        print("  📁 또는 폴더 구조:")
        print("    📂 training_data/")
        print("      📄 data.yaml")
        print("      📂 images/")
        print("      📂 labels/")
        data_path = '/content/training_data'  # 기본값
    
except ImportError:
    IN_COLAB = False
    print("💻 로컬 환경 감지")
    data_path = './data'  # 로컬 기본 경로

# 📦 필수 패키지 설치
packages = [
    "ultralytics",
    "torch", 
    "torchvision",
    "opencv-python",
    "pillow",
    "matplotlib",
    "numpy",
    "pandas",
    "roboflow"  # 데이터셋 관리용 추가
]

print(f"\n📦 필수 패키지 설치:")
for package in packages:
    install_package(package)

# 📚 라이브러리 임포트
import torch
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

print(f"\n🖥️ 시스템 정보:")
print(f"  Python: {sys.version.split()[0]}")
print(f"  PyTorch: {torch.__version__}")
print(f"  CUDA 사용가능: {'✅' if torch.cuda.is_available() else '❌'}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"  GPU: {gpu_name}")
    print(f"  GPU 메모리: {gpu_memory:.1f}GB")
    
    # GPU 메모리 정리
    torch.cuda.empty_cache()
    print("  🧹 GPU 메모리 정리 완료")
else:
    print("  ⚠️ GPU를 사용할 수 없습니다. 런타임 설정을 확인하세요!")

# 🏆 기존 학습된 모델 자동 탐지 (강화된 버전)
print(f"\n🔍 기존 학습된 모델 탐지:")

def find_best_model():
    """기존 학습된 모델을 찾는 향상된 함수"""
    # 우선순위별 모델 검색 경로
    search_priorities = [
        # 1순위: 최신 학습 결과 (피해목 탐지 전용)
        {
            'priority': 1,
            'description': '최신 피해목 탐지 모델',
            'patterns': [
                'pinetree_yolov11s/damage_detection_*/weights/best.pt',
                '/content/pinetree_yolov11s/damage_detection_*/weights/best.pt',
                '/content/drive/MyDrive/pinetree_scan/results/damage_detection_*/weights/best.pt',
                '/content/drive/MyDrive/best.pt'
            ]
        },
        # 2순위: 일반 학습 결과
        {
            'priority': 2,
            'description': '일반 학습 모델',
            'patterns': [
                'runs/detect/train*/weights/best.pt',
                '/content/runs/detect/train*/weights/best.pt',
                '/content/drive/MyDrive/pinetree_scan/runs/detect/train*/weights/best.pt',
            ]
        },
        # 3순위: 수동 저장 모델
        {
            'priority': 3,
            'description': '수동 저장 모델',
            'patterns': [
                '/content/drive/MyDrive/pinetree_scan/models/best.pt',
                '/content/drive/MyDrive/pinetree_scan/weights/best.pt',
                'models/best.pt',
                'weights/best.pt',
                '/content/best.pt'
            ]
        },
        # 4순위: 기본 모델
        {
            'priority': 4,
            'description': '기본 YOLO 모델',
            'patterns': [
                'yolo11s.pt',
                '/content/yolo11s.pt'
            ]
        }
    ]
    
    found_models = []
    
    for priority_group in search_priorities:
        for pattern in priority_group['patterns']:
            if '*' in pattern:
                matches = glob.glob(pattern)
                for match in matches:
                    if os.path.exists(match):
                        found_models.append({
                            'path': match,
                            'priority': priority_group['priority'],
                            'description': priority_group['description'],
                            'size': os.path.getsize(match) / 1024 / 1024,  # MB
                            'modified': os.path.getctime(match)
                        })
            else:
                if os.path.exists(pattern):
                    found_models.append({
                        'path': pattern,
                        'priority': priority_group['priority'],
                        'description': priority_group['description'],
                        'size': os.path.getsize(pattern) / 1024 / 1024,  # MB
                        'modified': os.path.getctime(pattern)
                    })
    
    if found_models:
        # 우선순위와 수정시간으로 정렬
        found_models.sort(key=lambda x: (x['priority'], -x['modified']))
        return found_models
    
    return None

# 모델 검색 실행
found_models = find_best_model()

if found_models:
    print(f"🎯 발견된 모델: {len(found_models)}개")
    
    # 상위 3개 모델 표시
    for i, model_info in enumerate(found_models[:3]):
        emoji = "🏆" if i == 0 else "🥈" if i == 1 else "🥉"
        print(f"  {emoji} {model_info['description']}")
        print(f"     경로: {model_info['path']}")
        print(f"     크기: {model_info['size']:.1f}MB")
        print(f"     수정: {datetime.fromtimestamp(model_info['modified']).strftime('%Y-%m-%d %H:%M')}")
        
        # 첫 번째 모델 선택
        if i == 0:
            selected_model = model_info['path']
    
    # 선택된 모델 로드 및 검증
    print(f"\n📥 선택된 모델 로드: {os.path.basename(selected_model)}")
    
    try:
        temp_model = YOLO(selected_model)
        
        # 모델 정보 확인
        print(f"✅ 모델 로드 성공!")
        print(f"  📊 모델 정보:")
        
        if hasattr(temp_model, 'model') and hasattr(temp_model.model, 'names'):
            class_names = list(temp_model.model.names.values())
            print(f"    🏷️ 클래스 수: {len(class_names)}")
            print(f"    🏷️ 클래스명: {class_names}")
            
            # 소나무/피해목 관련 클래스 확인
            pine_related = any(keyword in ' '.join(class_names).lower() 
                             for keyword in ['pine', 'damage', 'tree', '피해', '소나무', 'damaged'])
            
            if pine_related:
                print(f"    🌲 소나무 피해목 탐지용 모델 확인!")
            else:
                print(f"    ⚠️ 일반 객체 탐지 모델 (전이학습 예정)")
        
        # 모델 아키텍처 정보
        try:
            model_yaml = temp_model.model.yaml
            if 'backbone' in str(model_yaml):
                print(f"    🏗️ 아키텍처: YOLOv11s")
        except:
            print(f"    🏗️ 아키텍처: YOLO 계열")
            
        existing_model = selected_model
        del temp_model  # 메모리 정리
        
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        print(f"📥 기본 YOLOv11s 모델로 대체")
        existing_model = 'yolo11s.pt'

else:
    print("❌ 기존 학습 모델을 찾을 수 없습니다.")
    print("📥 기본 YOLOv11s 모델 사용")
    existing_model = 'yolo11s.pt'

# 🎯 모델 사용 계획 안내
print(f"\n🎯 모델 사용 계획:")
if 'yolo11s.pt' in existing_model:
    print(f"  📝 기본 YOLOv11s → 소나무 피해목 전이학습")
    print(f"  🎯 예상 학습 시간: 1-2시간 (처음부터)")
    print(f"  📈 예상 성능: 85-95% mAP50")
elif 'damage_detection' in existing_model:
    print(f"  📝 기존 피해목 모델 → 추가 학습/미세조정")
    print(f"  🎯 예상 학습 시간: 30분-1시간 (미세조정)")
    print(f"  📈 예상 성능: 90-98% mAP50")
else:
    print(f"  📝 기존 모델 → 소나무 피해목 전이학습")
    print(f"  🎯 예상 학습 시간: 1시간 (전이학습)")
    print(f"  📈 예상 성능: 87-95% mAP50")

# 🗂️ 출력 디렉토리 설정
if IN_COLAB:
    output_base = '/content/drive/MyDrive/pinetree_scan/results'
else:
    output_base = './results'

os.makedirs(output_base, exist_ok=True)
print(f"\n📁 결과 저장 경로: {output_base}")

# 📊 데이터셋 정보 확인
data_yaml_path = f"{data_path}/data.yaml"
if os.path.exists(data_yaml_path):
    print(f"\n📊 데이터셋 정보:")
    try:
        with open(data_yaml_path, 'r') as f:
            content = f.read()
            print(f"  📄 data.yaml 내용 확인됨")
            if 'names:' in content:
                print(f"  🏷️ 클래스 정보 포함됨")
    except:
        print(f"  ⚠️ data.yaml 읽기 실패")
else:
    print(f"\n⚠️ 데이터셋 파일 없음: {data_yaml_path}")
    print("📋 data.yaml 파일이 필요합니다!")

# 환경 변수 설정
globals()['existing_model'] = existing_model
globals()['data_path'] = data_path
globals()['output_base'] = output_base
globals()['IN_COLAB'] = IN_COLAB
globals()['found_models'] = found_models

print(f"\n✅ 환경 설정 완료!")
print(f"  🏆 사용할 모델: {os.path.basename(existing_model)}")
print(f"  📂 모델 경로: {existing_model}")
print(f"  📊 데이터 경로: {data_path}")
print(f"  📁 결과 경로: {output_base}")
print("=" * 50)
print("✅ 1단계 완료! 환경 설정 및 모델 탐지 완료")
print(f"📦 사용할 모델: {existing_model}")  
print(f"� 데이터 경로: {data_path}")
print(f"� 환경: {'Google Colab' if IN_COLAB else '로컬'}")
print("🔄 다음 단계: 2단계 멀티스케일 학습을 실행하세요!")

## 🚀 **2단계: YOLOv11s 멀티스케일 학습 + Google Drive 자동 백업**

> **멀티스케일 최적화로 mAP 0.65+ 목표 달성**  
> Tesla T4 GPU 최적화 설정으로 다양한 크기의 피해목을 효과적으로 탐지하는 모델을 학습합니다.  
> **자동 백업**: 학습 완료 후 모든 결과를 Google Drive에 자동 저장  
> **네이밍**: `damage_detection_YYYYMMDD_HHMM` 형식으로 자동 생성

In [ ]:
# 🚀 2단계: YOLOv11s Multi-Scale 학습 준비 + 실행 (T4 GPU 최적화)
print("🌲 YOLOv11s Multi-Scale 소나무 피해목 학습 시작!")
print("🎯 T4 GPU 최적화 + Multi-Scale Detection + 오탐지 개선 + 동적 바운딩박스")
print("=" * 60)

# 📚 필수 라이브러리 재임포트 (셀 독립 실행 지원)
import torch
from ultralytics import YOLO
import os
import yaml
import numpy as np
from datetime import datetime
import shutil
import json

print("✅ 라이브러리 임포트 완료")

# 🧹 메모리 사전 정리 (무한로딩 방지)
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 GPU 메모리 정리 완료")

# 🔍 1단계 연결 확인 (안전한 기본값)
existing_model = globals().get('existing_model', 'yolo11s.pt')
data_path = globals().get('data_path', '/content/drive/MyDrive/pinetree_scan/training_data')

print(f"📦 사용할 모델: {existing_model}")
print(f"📂 데이터 기본 경로: {data_path}")

# 📊 1단계에서 준비된 데이터셋 확인 및 절대 경로 수정
print("\n🔍 1단계 데이터셋 확인 및 절대 경로 수정 중...")

# 🎯 1단계에서 생성된 경로
original_yaml_path = '/content/drive/MyDrive/pinetree_scan/training_data/data.yaml'
data_base_dir = '/content/drive/MyDrive/pinetree_scan/training_data'

print(f"📂 원본 data.yaml: {original_yaml_path}")
print(f"📂 데이터 기본 경로: {data_base_dir}")

if not os.path.exists(original_yaml_path):
    print("❌ 1단계 data.yaml 파일이 없습니다!")
    print(f"📁 확인한 경로: {original_yaml_path}")
    print("\n💡 해결 방법:")
    print("1. 1단계 셀을 먼저 실행하여 데이터를 준비하세요")
    print("2. Google Drive 마운트를 확인하세요")
    raise FileNotFoundError("1단계 데이터가 준비되지 않았습니다")

# 📖 원본 data.yaml 읽기
try:
    with open(original_yaml_path, 'r', encoding='utf-8') as f:
        original_data = yaml.safe_load(f)
        
    print(f"📊 원본 data.yaml 내용:")
    print(f"  🏷️ 클래스: {original_data.get('names', ['정보 없음'])}")
    print(f"  📊 클래스 수: {original_data.get('nc', '정보 없음')}")
    print(f"  📁 경로: {original_data.get('path', '정보 없음')}")
    print(f"  🎯 훈련: {original_data.get('train', '정보 없음')}")
    print(f"  ✅ 검증: {original_data.get('val', '정보 없음')}")
    
except Exception as e:
    print(f"❌ 원본 data.yaml 읽기 실패: {e}")
    raise e

# 🔧 절대 경로로 수정된 data.yaml 생성
print(f"\n🔧 절대 경로 data.yaml 생성 중...")

# 실제 폴더 존재 확인
train_images_path = os.path.join(data_base_dir, 'train', 'images')
val_images_path = os.path.join(data_base_dir, 'val', 'images')
train_labels_path = os.path.join(data_base_dir, 'train', 'labels')
val_labels_path = os.path.join(data_base_dir, 'val', 'labels')

print(f"📂 확인할 경로들:")
print(f"  📈 훈련 이미지: {train_images_path}")
print(f"  📊 검증 이미지: {val_images_path}")
print(f"  🏷️ 훈련 라벨: {train_labels_path}")
print(f"  🏷️ 검증 라벨: {val_labels_path}")

# 필수 폴더 확인
missing_folders = []
if not os.path.exists(train_images_path):
    missing_folders.append("train/images")
if not os.path.exists(val_images_path):
    missing_folders.append("val/images")
if not os.path.exists(train_labels_path):
    missing_folders.append("train/labels")
if not os.path.exists(val_labels_path):
    missing_folders.append("val/labels")

if missing_folders:
    print(f"❌ 필수 폴더가 없습니다: {', '.join(missing_folders)}")
    print("\n💡 해결 방법:")
    print("1. 1단계를 다시 실행하세요")
    print("2. ZIP 파일이 올바르게 압축해제되었는지 확인하세요")
    raise FileNotFoundError(f"필수 폴더가 없습니다: {', '.join(missing_folders)}")

print("✅ 모든 필수 폴더 확인 완료!")

# 📊 실제 파일 수 확인
train_images = [f for f in os.listdir(train_images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.tiff'))]
val_images = [f for f in os.listdir(val_images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.tiff'))]
train_labels = [f for f in os.listdir(train_labels_path) if f.endswith('.txt')]
val_labels = [f for f in os.listdir(val_labels_path) if f.endswith('.txt')]

print(f"\n📈 실제 파일 수:")
print(f"  🖼️ 훈련 이미지: {len(train_images)}개")
print(f"  🖼️ 검증 이미지: {len(val_images)}개")
print(f"  🏷️ 훈련 라벨: {len(train_labels)}개")
print(f"  🏷️ 검증 라벨: {len(val_labels)}개")

# 🆕 절대 경로로 수정된 data.yaml 생성
fixed_data = {
    'path': data_base_dir,  # 절대 경로 루트
    'train': 'train/images',  # 상대 경로
    'val': 'val/images',     # 상대 경로
    'nc': original_data.get('nc', 1),
    'names': original_data.get('names', ['damaged_tree'])
}

# 임시 파일로 저장
fixed_yaml_path = '/tmp/fixed_training_data.yaml'
with open(fixed_yaml_path, 'w', encoding='utf-8') as f:
    yaml.dump(fixed_data, f, default_flow_style=False, allow_unicode=True)

print(f"🔧 수정된 data.yaml 생성: {fixed_yaml_path}")
print(f"📊 수정된 내용:")
print(f"  📁 루트 경로: {fixed_data['path']}")
print(f"  🎯 훈련: {fixed_data['train']}")
print(f"  ✅ 검증: {fixed_data['val']}")
print(f"  📊 클래스 수: {fixed_data['nc']}")
print(f"  🏷️ 클래스명: {fixed_data['names']}")

# 수정된 yaml 사용
data_yaml_path = fixed_yaml_path

# 🤖 1단계에서 설정한 기존 모델 사용
print(f"\n🤖 1단계에서 설정한 기존 모델 로드 중...")
print(f"📂 1단계 모델: {existing_model}")

# 1단계에서 설정한 모델 사용
model_path = existing_model

try:
    # 🔒 기존 모델만 사용 (추가 다운로드 방지)
    model = YOLO(model_path, task='detect')
    print(f"✅ 기존 YOLOv11s 모델 로드 성공! (다운로드 없음)")
    
    # 모델 정보 표시
    model_info = model.info(detailed=False)
    print(f"🚫 추가 다운로드 방지: AMP 체크 시 YOLOv11n 다운로드 차단됨")
    
    # 모델 타입 확인
    if 'best.pt' in model_path:
        print(f"📊 기존 학습된 모델 정보:")
        print(f"  🎯 모델: 기존 학습된 YOLO 모델 (전이학습)")
        print(f"  💡 장점: 이미 학습된 가중치로 빠른 수렴")
        print(f"  🔄 전략: Fine-tuning으로 성능 향상")
    else:
        print(f"📊 기본 YOLOv11s 모델 정보:")
        print(f"  🔢 파라미터: ~9.4M (메모리 효율적)")
        print(f"  🎯 mAP 목표: 65%+ (안정적 성능)")
        print(f"  💾 모델 크기: ~18MB")
        
    print(f"  📁 모델 경로: {model_path}")
    print(f"  💡 1단계 연동: 기존 설정 활용")
    
except Exception as e:
    print(f"❌ 기존 모델 로드 실패: {e}")
    print(f"💡 YOLOv11s 기본 모델로 대체...")
    model_path = 'yolo11s.pt'
    model = YOLO(model_path)  # 기본 모델로 대체
    print(f"✅ YOLOv11s 기본 모델 로드 완료")

# 🔥 T4 GPU 메모리 최적화 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🖥️ 컴퓨팅 장치: {device}")

if device == 'cuda':
    # T4 GPU 메모리 정보
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    print(f"  🚀 GPU: {gpu_name}")
    print(f"  💾 메모리: {gpu_memory:.1f}GB")
    
    # 메모리 효율적 배치 크기 (YOLOv11s + T4 최적화)
    if 'T4' in gpu_name:
        batch_size = 4   # T4 메모리 안정성 우선 (8→4)
    else:
        batch_size = 6   # 다른 GPU에서도 보수적
        
    print(f"📦 메모리 안정 배치 크기: {batch_size} (기존 8→4로 감소)")
    
    # 성능 최적화 설정
    torch.backends.cudnn.benchmark = True   # 성능 향상
    if hasattr(torch.backends.cudnn, 'allow_tf32'):
        torch.backends.cudnn.allow_tf32 = True  # T4 Tensor Core 활용
    
    if hasattr(torch, 'set_float32_matmul_precision'):
        torch.set_float32_matmul_precision('medium')  # 속도 vs 정밀도 균형
        
    # Multi-Scale 최적화 
    torch.backends.cudnn.deterministic = False  # 성능 우선
    
else:
    batch_size = 2  # CPU 모드
    print(f"📦 CPU 모드 배치 크기: {batch_size}")

# ================================
# 🎯 mAP 65% 달성 최적화 파라미터 설정 완료
# ================================
# 모든 하이퍼파라미터와 증강 설정이 아래 model.train() 함수에 직접 통합되었습니다.
# 재현율 향상을 위한 핵심 설정:
# - 학습률: 0.008 (적극적 학습)
# - 박스 손실: 10.0 (위치 정확도)
# - Mosaic: 1.0 + Copy-Paste: 0.5 (작은 객체 강화)
# - Patience: 80 (충분한 학습 시간)
print(f"\n🎯 재현율 최적화 하이퍼파라미터 설정 완료!")
print(f"📊 목표: mAP 42.3% → 65%+ (22.7% 향상)")

print(f"\n🚀 메모리 효율적 YOLOv11s 학습 설정:")
print(f"  🤖 모델: YOLOv11s (9.4M 파라미터) - 메모리 효율적")
print(f"  📦 배치 크기: {batch_size} (메모리 안정성 우선)")
print(f"  🔄 에폭: 200 (Early Stopping: 50 patience)")
print(f"  💾 장치: {device}")
print(f"  📏 이미지 크기: 640px")
print(f"  📂 수정된 데이터셋: {data_yaml_path}")
print(f"  📊 훈련 이미지: {len(train_images)}개")
print(f"  📊 검증 이미지: {len(val_images)}개")
print(f"")
print(f"? 메모리 최적화 전략:")
print(f"  📉 배치 크기 최적화: 8 → {batch_size} (T4 GPU 안정성)")
print(f"  📈 학습률 최적화: 0.005 → 0.008 (적극적 학습)")
print(f"  🎯 손실 가중치 강화: box=10.0, cls=1.0, dfl=2.0")
print(f"  🎨 데이터 증강 극대화:")
print(f"    - Mosaic: 1.0 (최대 다양성)")
print(f"    - Copy-Paste: 0.5 (작은 객체 복제)")
print(f"    - 회전 각도: 15° (다양한 각도)")
print(f"    - 크기 변화: 0.8 (극한 크기 변화)")
print(f"  ⏰ 충분한 학습: 80 에폭 patience (재현율 집중)")
print(f"")
print(f"🎯 mAP 65% 달성을 위한 재현율 집중 최적화:")
print(f"  • 🎯 mAP@0.5: 42.3% → 65%+ (22.7% 대폭 향상 목표)")
print(f"  • 🔄 재현율: 45.3% → 65%+ (놓친 피해목 20% 감소)")  
print(f"  • 🎯 정밀도: 56.4% → 70%+ (오탐지 추가 감소)")
print(f"  • ⚡ 핵심 전략: 작은 객체 탐지 강화")
print(f"  • 🎨 증강 강화: 다양한 변형으로 견고한 학습")
print(f"")
print(f"🔍 재현율 향상 핵심 최적화:")
print(f"  • 🎨 Mosaic 1.0 + Copy-Paste 0.5 (작은 객체 복제)")
print(f"  • 📈 학습률 0.008 (적극적 학습)")
print(f"  • 🎯 박스 손실 10.0 (위치 정확도)")
print(f"  • ⏰ Patience 80 (충분한 학습)")
print(f"  • 🔄 Close Mosaic 15 (더 오래 증강)")
print(f"")
print(f"💡 예상 학습 시간: 1.5-2.5시간 (고성능)")
print(f"🚀 목표 mAP 65% 달성으로 실용적 성능 확보!")

# 🎯 mAP 65% 달성 재현율 최적화 학습 실행
print(f"\n🚀 재현율 집중 YOLOv11s 고성능 학습 시작!")
print(f"🎯 목표: mAP 42.3% → 65%+ (22.7% 대폭 향상)")
print(f"📐 이미지 크기: 640x640 (고정)")
print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"")
print(f"🔍 재현율 향상 핵심 전략:")
print(f"  1️⃣ 강화된 데이터 증강 (Mosaic 1.0, Copy-Paste 0.5)")
print(f"  2️⃣ 높은 학습률 (0.008) + 긴 워밍업 (15 에폭)")
print(f"  3️⃣ 박스 손실 강화 (10.0) + 클래스 손실 (1.0)")
print(f"  4️⃣ 충분한 학습 시간 (Patience 80)")
print(f"  5️⃣ 드롭아웃 0.1로 일반화 향상")
print(f"")

try:
    # 🎯 mAP 65% 달성을 위한 고성능 학습 실행
    results = model.train(
        # ================================
        # 📂 기본 설정
        # ================================
        data=data_yaml_path,              # 데이터셋 경로
        epochs=200,                       # 학습 에폭 수
        batch=batch_size,                 # 배치 크기 (T4 GPU 최적화)
        imgsz=640,                        # 이미지 크기 (640x640 고정)
        device=device,                    # GPU/CPU 설정
        
        # ================================
        # 📊 학습 모니터링 및 저장
        # ================================
        patience=80,                      # Early Stopping (충분한 학습 시간)
        save=True,                        # 모델 저장 활성화
        save_period=10,                   # 10 에폭마다 체크포인트
        project='runs/detect',            # 결과 저장 디렉토리
        name=f'pinetree_optimized_{datetime.now().strftime("%Y%m%d_%H%M")}',
        exist_ok=True,                    # 기존 폴더 덮어쓰기
        
        # ================================
        # 📈 재현율 향상 학습률 설정
        # ================================
        lr0=0.008,                        # 초기 학습률 (적극적 학습)
        lrf=0.0001,                       # 최종 학습률 (세밀한 조정)
        momentum=0.95,                    # 모멘텀 (안정적 수렴)
        weight_decay=0.0003,              # 가중치 감쇠 (과적합 방지)
        
        # ================================
        # 🔥 워밍업 최적화
        # ================================
        warmup_epochs=15.0,               # 워밍업 에폭 (안정적 시작)
        warmup_momentum=0.9,              # 워밍업 모멘텀
        warmup_bias_lr=0.01,              # 워밍업 바이어스 학습률
        
        # ================================
        # 🎯 손실 함수 가중치 (재현율 최적화)
        # ================================
        box=10.0,                         # 박스 손실 (위치 정확도 강화)
        cls=1.0,                          # 클래스 손실 (분류 정확도)
        dfl=2.0,                          # DFL 손실 (세밀한 경계 학습)
        
        # ================================
        # 🌈 색상 증강 (다양한 조건 대응)
        # ================================
        hsv_h=0.025,                      # 색조 변화 (계절별 변화)
        hsv_s=0.8,                        # 채도 변화 (밝기 변화)
        hsv_v=0.5,                        # 명도 변화 (조명 변화)
        
        # ================================
        # 🎨 재현율 향상 데이터 증강
        # ================================
        mosaic=1.0,                       # Mosaic 최대화 (다양한 조합)
        mixup=0.3,                        # MixUp (경계 학습 강화)
        copy_paste=0.5,                   # Copy-Paste (작은 객체 복제)
        
        # ================================
        # 🔄 기하학적 변환 (다양한 상황 대응)
        # ================================
        degrees=15.0,                     # 회전 각도 (다양한 각도)
        translate=0.2,                    # 이동 범위 (위치 변화)
        scale=0.8,                        # 크기 변화 (극한 크기)
        shear=3.0,                        # 전단 변형 (변형된 형태)
        perspective=0.0005,               # 원근 변환 (거리감)
        flipud=0.0,                       # 상하 반전 비활성화
        fliplr=0.7,                       # 좌우 반전 (대칭성)
        erasing=0.2,                      # 랜덤 지우기 (정보 보존)
        
        # ================================
        # 🧠 정규화 및 최적화
        # ================================
        dropout=0.1,                      # 드롭아웃 (일반화 향상)
        nbs=64,                           # 정규화 배치 크기
        overlap_mask=True,                # 겹침 마스크 (중복 방지)
        mask_ratio=4,                     # 마스크 비율 (세밀한 탐지)
        
        # ================================
        # ⚙️ 시스템 최적화
        # ================================
        workers=2,                        # 데이터 로더 워커 (메모리 절약)
        verbose=True,                     # 상세 로그
        seed=42,                          # 재현 가능한 결과
        deterministic=False,              # 성능 우선
        single_cls=True,                  # 단일 클래스 (소나무만)
        rect=False,                       # 다양한 비율 학습
        cos_lr=True,                      # 코사인 학습률 스케줄러
        close_mosaic=15,                  # Mosaic 비활성화 시점 (더 오래 증강)
        resume=False,                     # 새로 시작
        amp=False,                        # AMP 비활성화 (YOLOv11n 다운로드 방지)
        fraction=1.0,                     # 전체 데이터셋 사용
        profile=False,                    # 프로파일링 비활성화
        cache=False,                      # 캐시 비활성화 (메모리 절약)
    )
    
    print(f"✅ 크기별 소나무 탐지 YOLOv11s 학습 완료!")
    print(f"🌲 다양한 크기 소나무 탐지 모델 학습 성공!")
    print(f"⏰ 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 📊 학습 결과 요약
    if results:
        print(f"\n📊 학습 결과 요약:")
        
        # 최종 메트릭 추출
        final_metrics = results.results_dict if hasattr(results, 'results_dict') else {}
        
        if final_metrics:
            map50 = final_metrics.get('metrics/mAP50(B)', 'N/A')
            map50_95 = final_metrics.get('metrics/mAP50-95(B)', 'N/A') 
            precision = final_metrics.get('metrics/precision(B)', 'N/A')
            recall = final_metrics.get('metrics/recall(B)', 'N/A')
            
            print(f"  📈 mAP@0.5: {map50}")
            print(f"  📈 mAP@0.5:0.95: {map50_95}")
            print(f"  🎯 정밀도: {precision}")
            print(f"  🔄 재현율: {recall}")
        
        # 모델 저장 경로 확인
        save_dir = results.save_dir if hasattr(results, 'save_dir') else 'runs/detect/train'
        best_model = os.path.join(save_dir, 'weights', 'best.pt')
        last_model = os.path.join(save_dir, 'weights', 'last.pt')
        
        print(f"\n💾 모델 저장 위치:")
        print(f"  🥇 최고 모델: {best_model}")
        print(f"  📝 최종 모델: {last_model}")
        
        # 🚀 Google Drive 전체 딥러닝 결과 백업
        try:
            print(f"\n☁️ Google Drive 전체 결과 백업 시작...")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            # 🗂️ 백업 디렉토리 구조 생성
            main_backup_dir = '/content/drive/MyDrive/pinetree_scan/training_results'
            session_backup_dir = os.path.join(main_backup_dir, f'training_session_{timestamp}')
            models_backup_dir = os.path.join(session_backup_dir, 'models')
            results_backup_dir = os.path.join(session_backup_dir, 'results')
            logs_backup_dir = os.path.join(session_backup_dir, 'logs')
            
            os.makedirs(models_backup_dir, exist_ok=True)
            os.makedirs(results_backup_dir, exist_ok=True) 
            os.makedirs(logs_backup_dir, exist_ok=True)
            
            print(f"📁 백업 디렉토리 생성: {session_backup_dir}")
            
            # 🤖 모델 파일 백업
            model_backup_count = 0
            if os.path.exists(best_model):
                backup_best = os.path.join(models_backup_dir, f'best_model_{timestamp}.pt')
                shutil.copy2(best_model, backup_best)
                print(f"  🥇 최고 모델 백업: {os.path.basename(backup_best)}")
                model_backup_count += 1
            
            if os.path.exists(last_model):
                backup_last = os.path.join(models_backup_dir, f'last_model_{timestamp}.pt')
                shutil.copy2(last_model, backup_last)
                print(f"  📝 최종 모델 백업: {os.path.basename(backup_last)}")
                model_backup_count += 1
            
            # 📊 학습 결과 파일들 백업
            results_backup_count = 0
            if os.path.exists(save_dir):
                # 결과 이미지들 백업
                for img_file in ['confusion_matrix.png', 'results.png', 'labels.jpg', 'val_batch0_pred.jpg']:
                    img_path = os.path.join(save_dir, img_file)
                    if os.path.exists(img_path):
                        backup_img = os.path.join(results_backup_dir, f'{timestamp}_{img_file}')
                        shutil.copy2(img_path, backup_img)
                        results_backup_count += 1
                
                # CSV 결과 파일 백업
                for csv_file in ['results.csv']:
                    csv_path = os.path.join(save_dir, csv_file)
                    if os.path.exists(csv_path):
                        backup_csv = os.path.join(logs_backup_dir, f'{timestamp}_{csv_file}')
                        shutil.copy2(csv_path, backup_csv)
                        results_backup_count += 1
                
                print(f"  📊 결과 파일 백업: {results_backup_count}개 파일")
            
            # 📈 학습 요약 정보 저장
            summary_info = {
                'training_timestamp': timestamp,
                'model_type': 'YOLOv11s',
                'training_images': len(train_images) if 'train_images' in globals() else 'N/A',
                'validation_images': len(val_images) if 'val_images' in globals() else 'N/A',
                'batch_size': batch_size if 'batch_size' in globals() else 'N/A',
                'epochs': 200,
                'device': device if 'device' in globals() else 'N/A',
                'final_metrics': final_metrics if 'final_metrics' in locals() else {}
            }
            
            summary_path = os.path.join(session_backup_dir, f'training_summary_{timestamp}.json')
            with open(summary_path, 'w', encoding='utf-8') as f:
                json.dump(summary_info, f, indent=2, ensure_ascii=False, default=str)
            
            print(f"  📋 학습 요약 저장: training_summary_{timestamp}.json")
            
            # 🎯 백업 완료 요약
            print(f"\n✅ Google Drive 전체 딥러닝 결과 백업 완료!")
            print(f"📁 백업 위치: {session_backup_dir}")
            print(f"🤖 모델 파일: {model_backup_count}개")
            print(f"📊 결과 파일: {results_backup_count}개")
            print(f"📋 요약 정보: 1개")
            print(f"💾 총 백업 크기: 약 {sum(os.path.getsize(os.path.join(root, f)) for root, dirs, files in os.walk(session_backup_dir) for f in files) / 1024 / 1024:.1f}MB")
            
        except Exception as backup_error:
            print(f"❌ Google Drive 백업 실패: {backup_error}")
            print(f"💡 수동 백업 필요:")
            print(f"  🥇 최고 모델: {best_model}")
            print(f"  📝 최종 모델: {last_model}")
            print(f"  📊 결과 폴더: {save_dir}")
        
        # 🔗 전역 변수로 결과 저장 (3단계 연동)
        globals()['multiscale_results'] = results
        globals()['best_model_path'] = best_model
        globals()['last_model_path'] = last_model
        globals()['training_save_dir'] = save_dir
        globals()['model_trained'] = True
        globals()['trained_model_path'] = best_model
        globals()['training_timestamp'] = timestamp if 'timestamp' in locals() else datetime.now().strftime("%Y%m%d_%H%M%S")
        
        print(f"\n🎉 크기별 소나무 탐지 YOLOv11s 학습 대성공!")
        print(f"🌲 작은 소나무부터 큰 소나무까지 완벽 탐지!")
        print(f"☁️ Google Drive 전체 결과 백업 완료!")
        print(f"🎯 3단계 성능 분석을 위해 다음 셀을 실행하세요")
    
except Exception as e:
    print(f"❌ 학습 실패: {e}")
    print(f"💡 해결 방안:")
    print(f"  1. GPU 메모리 부족: 배치 크기를 더 줄여보세요 (현재: {batch_size})")
    print(f"  2. 데이터 경로 오류: 1단계를 다시 실행하세요")  
    print(f"  3. 모델 다운로드 실패: 인터넷 연결을 확인하세요")
    print(f"  4. 권한 문제: Google Drive 권한을 확인하세요")
    
    # 디버깅 정보
    print(f"\n🔍 디버깅 정보:")
    print(f"  📂 수정된 data.yaml: {data_yaml_path}")
    print(f"  📂 원본 data.yaml: {original_yaml_path}")
    print(f"  📂 데이터 기본 경로: {data_base_dir}")
    print(f"  🤖 모델 경로: {model_path}")
    print(f"  💾 장치: {device}")
    print(f"  📦 배치 크기: {batch_size}")
    
    raise e

print("\n" + "="*60)
print("🌲 2단계: 메모리 효율적 YOLOv11s 학습 완료!")
print("🎯 안정적 성능 향상 + GPU 메모리 최적화 적용됨")
print("📊 3단계 성능 분석 셀로 이동하세요!")
print("="*60)

## 🔍 **3단계: 학습된 모델 성능 테스트 및 시각화**

> **학습 완료된 모델로 실제 이미지에서 피해목 탐지 테스트**  
> 훈련 데이터셋의 일부 이미지를 사용하여 탐지 성능을 시각적으로 확인합니다.  
> **결과**: 바운딩박스 마킹된 탐지 결과 이미지 및 성능 분석

In [ ]:
# 🔍 3단계: 학습된 모델 성능 테스트 및 탐지 결과 시각화
print("🔍 3단계: 학습된 모델 성능 테스트 및 시각화")
print("🎯 훈련된 모델로 실제 피해목 탐지 및 시각적 확인")
print("=" * 60)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np
import random
import glob
from datetime import datetime

# 🧹 GPU 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 GPU 메모리 정리 완료")

# 🔍 2단계 학습 완료 상태 및 모델 경로 확인
model_trained = globals().get('model_trained', False)
trained_model_path = globals().get('trained_model_path', None)
data_yaml_path = globals().get('data_yaml_path', None)
save_dir = globals().get('save_dir', None)

print("📋 2단계 학습 상태 확인:")
print(f"  ✅ 모델 학습 완료: {'Yes' if model_trained else 'No'}")
print(f"  🤖 모델 경로: {trained_model_path or '없음'}")
print(f"  📊 데이터 경로: {data_yaml_path or '없음'}")
print(f"  📁 결과 디렉토리: {save_dir or '없음'}")

# 🎯 2단계에서 완료된 모델 직접 사용
def get_trained_model_from_step2():
    """2단계에서 학습 완료된 모델 경로 직접 가져오기"""
    print("\n🎯 2단계 완료 모델 경로 확인 중...")
    
    # 1순위: 2단계에서 저장된 전역변수 모델 경로
    if trained_model_path and os.path.exists(trained_model_path):
        print(f"✅ 2단계 완료 모델 발견: {os.path.basename(trained_model_path)}")
        print(f"  📁 모델 경로: {trained_model_path}")
        
        model_size = os.path.getsize(trained_model_path) / 1024 / 1024  # MB
        print(f"  📏 모델 크기: {model_size:.1f}MB")
        print(f"  🕒 수정 시간: {datetime.fromtimestamp(os.path.getctime(trained_model_path)).strftime('%Y-%m-%d %H:%M')}")
        
        return trained_model_path
    
    # 2순위: 2단계 결과 디렉토리에서 best.pt 찾기
    if save_dir:
        best_pt_path = os.path.join(save_dir, 'weights', 'best.pt')
        if os.path.exists(best_pt_path):
            print(f"✅ 2단계 결과 폴더에서 모델 발견: best.pt")
            print(f"  📁 모델 경로: {best_pt_path}")
            
            model_size = os.path.getsize(best_pt_path) / 1024 / 1024  # MB
            print(f"  📏 모델 크기: {model_size:.1f}MB")
            print(f"  🕒 수정 시간: {datetime.fromtimestamp(os.path.getctime(best_pt_path)).strftime('%Y-%m-%d %H:%M')}")
            
            return best_pt_path
    
    print("❌ 2단계 완료 모델을 찾을 수 없습니다")
    print("💡 해결 방법:")
    print("  1. 2단계 학습을 먼저 완료하세요")
    print("  2. 학습이 완료되었다면 노트북을 재시작하지 마세요")
    print("  3. 수동으로 모델 경로를 지정하세요")
    
    return None

# 🔍 백업 모델 탐지 (2단계 실패 시)
def find_backup_trained_model():
    """2단계 모델이 없을 때 백업 탐지"""
    print("\n🔍 백업 모델 탐지 중...")
    
    # 가능한 백업 경로들
    backup_search_paths = [
        '/content/runs/detect/pinetree_multiscale_*/weights/best.pt',
        '/content/runs/detect/train*/weights/best.pt', 
        '/content/drive/MyDrive/pinetree_scan/models/multiscale_yolo11s/best_multiscale_*.pt',
        '/content/drive/MyDrive/pinetree_scan/training_results/*/weights/best.pt'
    ]
    
    found_models = []
    
    for path_pattern in backup_search_paths:
        matches = glob.glob(path_pattern)
        found_models.extend(matches)
    
    if found_models:
        # 가장 최신 모델 선택
        latest_model = max(found_models, key=os.path.getctime)
        model_size = os.path.getsize(latest_model) / 1024 / 1024  # MB
        
        print(f"✅ 백업 모델 발견: {os.path.basename(latest_model)}")
        print(f"  📁 전체 경로: {latest_model}")
        print(f"  📏 모델 크기: {model_size:.1f}MB")
        print(f"  🕒 수정 시간: {datetime.fromtimestamp(os.path.getctime(latest_model)).strftime('%Y-%m-%d %H:%M')}")
        
        return latest_model
    else:
        print("❌ 백업 모델도 찾을 수 없습니다")
        return None

# 📂 2단계 데이터에서 테스트용 이미지 직접 가져오기
def get_test_images_from_step2():
    """2단계에서 사용된 데이터셋에서 테스트 이미지 직접 가져오기"""
    print("\n📂 2단계 데이터셋에서 테스트 이미지 선택 중...")
    
    # 🔍 실제 훈련에 사용된 데이터셋 경로 탐지 (우선순위 순)
    search_paths = []
    
    # 1순위: data.yaml에서 실제 경로 읽기
    if data_yaml_path and os.path.exists(data_yaml_path):
        print(f"✅ 2단계 데이터셋 YAML: {data_yaml_path}")
        
        # data.yaml 파일 읽어서 실제 경로 확인
        try:
            with open(data_yaml_path, 'r', encoding='utf-8') as f:
                yaml_content = f.read()
                print(f"📄 data.yaml 내용 확인:")
                for line in yaml_content.split('\n')[:10]:  # 첫 10줄만 출력
                    if line.strip():
                        print(f"    {line}")
                        
                # path 또는 train/val 경로 추출 시도
                for line in yaml_content.split('\n'):
                    if 'path:' in line and not line.strip().startswith('#'):
                        yaml_path = line.split('path:')[1].strip().strip('"\'')
                        if yaml_path:
                            search_paths.append(f"{yaml_path}/val/images")
                            search_paths.append(f"{yaml_path}/train/images")
                            print(f"  🎯 YAML에서 경로 발견: {yaml_path}")
                    elif ('train:' in line or 'val:' in line) and not line.strip().startswith('#'):
                        # 절대 경로 추출
                        path_part = line.split(':')[1].strip().strip('"\'')
                        if path_part and os.path.isabs(path_part):
                            search_paths.append(path_part)
                            print(f"  🎯 YAML에서 절대 경로 발견: {path_part}")
        except Exception as e:
            print(f"  ⚠️ data.yaml 읽기 실패: {str(e)[:50]}")
    
    # 2순위: data_yaml_path 기준 경로들
    if data_yaml_path:
        dataset_dir = os.path.dirname(data_yaml_path)
        search_paths.extend([
            f"{dataset_dir}/val/images",
            f"{dataset_dir}/train/images",
            f"{dataset_dir}/test/images"
        ])
    
    # 3순위: 일반적인 완료 훈련 데이터셋 경로들
    search_paths.extend([
        "/content/complete_training_*/training_dataset/val/images",
        "/content/complete_training_*/training_dataset/train/images", 
        "/content/complete_training_*/training_dataset/test/images",
        "/content/drive/MyDrive/pinetree_scan/training_data/val/images",
        "/content/drive/MyDrive/pinetree_scan/training_data/train/images"
    ])
    
    print(f"🔍 탐지할 경로들: {len(search_paths)}개")
    
    # 경로별로 이미지 검색 (상세 디버깅 포함)
    all_images = []
    found_paths = []
    
    for i, search_path in enumerate(search_paths, 1):
        print(f"  🔍 [{i}/{len(search_paths)}] 탐색 중: {search_path}")
        
        if '*' in search_path:
            # 와일드카드 경로 처리
            for ext in ['jpg', 'png', 'jpeg', 'tif', 'tiff']:
                pattern = f"{search_path}/*.{ext}"
                matches = glob.glob(pattern)
                if matches:
                    all_images.extend(matches)
                    found_paths.append(pattern)
                    print(f"    ✅ {ext.upper()}: {len(matches)}개 발견")
                    # 처음 몇 개 파일명 샘플 출력
                    for j, match in enumerate(matches[:3], 1):
                        print(f"      {j}. {os.path.basename(match)}")
                    if len(matches) > 3:
                        print(f"      ... 외 {len(matches)-3}개")
                else:
                    print(f"    ❌ {ext.upper()}: 없음")
        else:
            # 직접 경로 처리
            print(f"    📁 경로 존재 확인: {os.path.exists(search_path)}")
            if os.path.exists(search_path):
                # 해당 폴더의 모든 내용 확인
                try:
                    contents = os.listdir(search_path)
                    print(f"    📋 폴더 내용: {len(contents)}개 항목")
                    if contents:
                        image_files = [f for f in contents if f.lower().endswith(('.jpg', '.png', '.jpeg', '.tif', '.tiff'))]
                        if image_files:
                            print(f"    🖼️ 이미지 파일: {len(image_files)}개")
                            # 실제 full path로 추가
                            for img_file in image_files:
                                full_path = os.path.join(search_path, img_file)
                                all_images.append(full_path)
                            found_paths.append(search_path)
                            # 처음 몇 개 파일명 출력
                            for j, img_file in enumerate(image_files[:3], 1):
                                print(f"      {j}. {img_file}")
                            if len(image_files) > 3:
                                print(f"      ... 외 {len(image_files)-3}개")
                        else:
                            print(f"    ❌ 이미지 파일 없음")
                            # 다른 파일들이 뭐가 있는지 확인
                            other_files = contents[:5]
                            if other_files:
                                print(f"    📄 다른 파일들: {', '.join(other_files)}")
                    else:
                        print(f"    📂 빈 폴더")
                except Exception as e:
                    print(f"    ❌ 폴더 읽기 오류: {str(e)}")
            else:
                print(f"    ❌ 경로 없음")
        
        print()  # 빈 줄로 구분
    
    if all_images:
        # 중복 제거
        unique_images = list(set(all_images))
        print(f"\n✅ 총 발견된 테스트 이미지: {len(unique_images)}개")
        print(f"  📂 검색된 경로: {len(found_paths)}개")
        
        # 검증용 이미지 우선 선택 (val > test > train 순)
        val_images = [img for img in unique_images if '/val/' in img]
        test_images = [img for img in unique_images if '/test/' in img]
        train_images = [img for img in unique_images if '/train/' in img]
        
        selected_images = []
        
        # 우선순위에 따라 선택
        if val_images:
            count = min(6, len(val_images))
            selected_images.extend(random.sample(val_images, count))
            print(f"  🎯 검증용(val) 이미지에서 {count}개 선택")
        
        remaining_slots = 8 - len(selected_images)
        if remaining_slots > 0 and test_images:
            count = min(remaining_slots, len(test_images))
            selected_images.extend(random.sample(test_images, count))
            print(f"  🎯 테스트(test) 이미지에서 {count}개 추가 선택")
        
        remaining_slots = 8 - len(selected_images)
        if remaining_slots > 0 and train_images:
            count = min(remaining_slots, len(train_images))
            selected_images.extend(random.sample(train_images, count))
            print(f"  🎯 훈련용(train) 이미지에서 {count}개 추가 선택")
        
        if selected_images:
            print(f"\n🎯 최종 선택된 테스트 이미지: {len(selected_images)}개")
            for i, img_path in enumerate(selected_images, 1):
                print(f"    {i}. {os.path.basename(img_path)} ({'val' if '/val/' in img_path else 'test' if '/test/' in img_path else 'train'})")
            
            return selected_images
    
    print("⚠️ 2단계 데이터를 찾을 수 없어 백업 경로에서 탐지합니다...")
    return find_backup_test_images()

# 📂 백업 테스트 이미지 탐지
def find_backup_test_images():
    """2단계 데이터가 없을 때 백업 이미지 탐지"""
    print("\n📂 백업 테스트 이미지 탐지 중...")
    
    # 가능한 백업 이미지 경로들 (모든 이미지 형식 포함)
    backup_base_paths = [
        '/content/drive/MyDrive/pinetree_scan/training_data/val/images',
        '/content/drive/MyDrive/pinetree_scan/training_data/test/images', 
        '/content/drive/MyDrive/pinetree_scan/training_data/train/images',
        '/content/complete_training_*/training_dataset/val/images',
        '/content/complete_training_*/training_dataset/train/images',
    ]
    
    all_images = []
    
    for base_path in backup_base_paths:
        for ext in ['jpg', 'png', 'jpeg', 'tif', 'tiff']:
            pattern = f"{base_path}/*.{ext}"
            matches = glob.glob(pattern)
            if matches:
                all_images.extend(matches)
                print(f"  ✅ {pattern}: {len(matches)}개 발견")
    
    if all_images:
        # 중복 제거
        unique_images = list(set(all_images))
        print(f"✅ 백업 테스트 이미지 발견: {len(unique_images)}개")
        
        # 랜덤하게 6-8개 선택
        test_count = min(8, len(unique_images))
        selected_images = random.sample(unique_images, test_count)
        
        print(f"  📋 선택된 이미지: {test_count}개")
        for i, img_path in enumerate(selected_images, 1):
            print(f"    {i}. {os.path.basename(img_path)}")
        
        return selected_images
    else:
        print("❌ 테스트용 이미지를 찾을 수 없습니다")
        return []

# 🎨 탐지 결과 시각화 함수 (Google Colab 최적화)
def visualize_detection_results(model, image_paths, save_dir):
    """탐지 결과 시각화 및 저장 (Colab에서 바로 확인 가능)"""
    print(f"\n🎨 탐지 결과 시각화 중...")
    
    detection_results = []
    visualization_paths = []
    
    # 시각화 결과 저장 디렉토리
    viz_dir = os.path.join(save_dir, 'detection_visualizations')
    os.makedirs(viz_dir, exist_ok=True)
    
    # matplotlib 한글 폰트 설정 (Colab용)
    plt.rcParams['font.family'] = ['DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
    
    for i, image_path in enumerate(image_paths, 1):
        try:
            print(f"  📸 {i}/{len(image_paths)}: {os.path.basename(image_path)} 처리 중...")
            
            # 이미지 추론
            results = model(image_path, conf=0.25, iou=0.6)
            
            # 원본 이미지 로드
            image = cv2.imread(image_path)
            if image is None:
                print(f"    ❌ 이미지 로드 실패: {image_path}")
                continue
                
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            height, width = image_rgb.shape[:2]
            
            # 탐지 결과 추출
            detections = []
            if len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes
                for box in boxes:
                    # 바운딩박스 좌표 (xyxy 형식)
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = float(box.conf[0])
                    class_id = int(box.cls[0])
                    
                    detections.append({
                        'bbox': [x1, y1, x2, y2],
                        'confidence': confidence,
                        'class_id': class_id
                    })
            
            # 🎨 시각화 생성 (Colab 최적화)
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
            
            # 원본 이미지 (왼쪽)
            ax1.imshow(image_rgb)
            ax1.set_title(f'Original: {os.path.basename(image_path)}', fontsize=12, fontweight='bold')
            ax1.set_xticks([])
            ax1.set_yticks([])
            
            # 탐지 결과 이미지 (오른쪽)
            ax2.imshow(image_rgb)
            ax2.set_title(f'Detection Result: {len(detections)} damaged trees found', 
                         fontsize=12, fontweight='bold', color='red' if detections else 'green')
            
            # 탐지된 객체에 바운딩박스 그리기
            colors = ['red', 'blue', 'lime', 'orange', 'purple', 'yellow', 'pink', 'cyan']
            
            for j, detection in enumerate(detections):
                x1, y1, x2, y2 = detection['bbox']
                confidence = detection['confidence']
                
                # 바운딩박스 색상
                color = colors[j % len(colors)]
                
                # 바운딩박스 그리기
                rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                       linewidth=4, edgecolor=color, facecolor='none', alpha=0.8)
                ax2.add_patch(rect)
                
                # 신뢰도 텍스트 (더 크고 명확하게)
                label = f'Damaged Tree\\nConfidence: {confidence:.3f}'
                ax2.text(x1, y1-20, label, fontsize=9, fontweight='bold',
                       color='white', 
                       bbox=dict(boxstyle="round,pad=0.5", facecolor=color, alpha=0.9, edgecolor='white'))
            
            ax2.set_xticks([])
            ax2.set_yticks([])
            
            # 전체 결과 정보
            fig.suptitle(f'🎯 Pine Tree Damage Detection Test #{i}\\n' + 
                        f'Image: {width}x{height}px | Detections: {len(detections)} | ' +
                        f'Avg Confidence: {np.mean([d["confidence"] for d in detections]):.3f}' if detections else f'No detections',
                        fontsize=14, fontweight='bold', y=0.95)
            
            plt.tight_layout()
            plt.subplots_adjust(top=0.85)
            
            # 🔍 Colab에서 바로 이미지 표시
            plt.show()
            
            # 결과 이미지 저장
            save_path = os.path.join(viz_dir, f'detection_result_{i:02d}_{os.path.basename(image_path)}.png')
            fig.savefig(save_path, dpi=150, bbox_inches='tight', facecolor='white')
            plt.close(fig)  # 메모리 정리
            
            visualization_paths.append(save_path)
            
            # 결과 정보 저장
            detection_results.append({
                'image_path': image_path,
                'image_name': os.path.basename(image_path),
                'detections': len(detections),
                'avg_confidence': np.mean([d['confidence'] for d in detections]) if detections else 0,
                'detection_details': detections,
                'visualization_path': save_path
            })
            
            print(f"    ✅ 탐지 완료: {len(detections)}개 피해목 발견 (신뢰도: {np.mean([d['confidence'] for d in detections]):.3f})" if detections else "    ⚪ 탐지된 피해목 없음")
            
            # 각 이미지별 구분선
            print("    " + "-"*50)
            
        except Exception as e:
            print(f"    ❌ 처리 실패: {str(e)}")
            import traceback
            traceback.print_exc()
            continue
    
    return detection_results, visualization_paths

# 📊 탐지 성능 분석 및 요약
def analyze_detection_performance(detection_results):
    """탐지 성능 분석 및 통계 요약"""
    print(f"\n📊 탐지 성능 분석:")
    
    if not detection_results:
        print("❌ 분석할 탐지 결과가 없습니다")
        return
    
    # 통계 계산
    total_images = len(detection_results)
    total_detections = sum(r['detections'] for r in detection_results)
    images_with_detections = sum(1 for r in detection_results if r['detections'] > 0)
    
    confidences = []
    for result in detection_results:
        for detection in result['detection_details']:
            confidences.append(detection['confidence'])
    
    print(f"  📸 총 테스트 이미지: {total_images}개")
    print(f"  🎯 총 탐지된 피해목: {total_detections}개")
    print(f"  ✅ 피해목 발견된 이미지: {images_with_detections}개 ({images_with_detections/total_images*100:.1f}%)")
    print(f"  📊 이미지당 평균 탐지: {total_detections/total_images:.2f}개")
    
    if confidences:
        avg_conf = np.mean(confidences)
        min_conf = np.min(confidences)
        max_conf = np.max(confidences)
        
        print(f"  🎯 신뢰도 통계:")
        print(f"    평균: {avg_conf:.3f}")
        print(f"    최소: {min_conf:.3f}")
        print(f"    최대: {max_conf:.3f}")
        
        # 신뢰도 분포
        high_conf = sum(1 for c in confidences if c >= 0.7)
        medium_conf = sum(1 for c in confidences if 0.5 <= c < 0.7)
        low_conf = sum(1 for c in confidences if c < 0.5)
        
        print(f"  📈 신뢰도 분포:")
        print(f"    높음 (≥0.7): {high_conf}개 ({high_conf/len(confidences)*100:.1f}%)")
        print(f"    중간 (0.5-0.7): {medium_conf}개 ({medium_conf/len(confidences)*100:.1f}%)")
        print(f"    낮음 (<0.5): {low_conf}개 ({low_conf/len(confidences)*100:.1f}%)")
    
    # 성능 평가
    if images_with_detections/total_images >= 0.8:
        performance_grade = "🏆 우수"
        performance_desc = "대부분 이미지에서 피해목을 성공적으로 탐지"
    elif images_with_detections/total_images >= 0.6:
        performance_grade = "✅ 양호"  
        performance_desc = "많은 이미지에서 피해목 탐지 성공"
    elif images_with_detections/total_images >= 0.4:
        performance_grade = "⚠️ 보통"
        performance_desc = "일부 이미지에서 탐지 성공, 개선 여지 있음"
    else:
        performance_grade = "❌ 미흡"
        performance_desc = "탐지 성능이 부족, 추가 학습 필요"
    
    print(f"\n🎯 성능 평가: {performance_grade}")
    print(f"  📝 평가: {performance_desc}")
    
    return {
        'total_images': total_images,
        'total_detections': total_detections, 
        'detection_rate': images_with_detections/total_images,
        'avg_confidence': np.mean(confidences) if confidences else 0,
        'performance_grade': performance_grade
    }

# 🚀 메인 실행 로직
print("🚀 성능 테스트 시작!")

try:
    # 1️⃣ 2단계 완료 모델 직접 사용
    best_model_path = get_trained_model_from_step2()
    
    # 2단계 모델이 없으면 백업 탐지
    if not best_model_path:
        print("\n⚠️ 2단계 모델을 찾을 수 없어 백업 모델을 탐지합니다...")
        best_model_path = find_backup_trained_model()
    
    if not best_model_path:
        print("❌ 사용할 수 있는 모델을 찾을 수 없어 테스트를 중단합니다")
        print("💡 해결 방법:")
        print("  1. 2단계 학습을 먼저 완료하세요")
        print("  2. 학습 완료 후 노트북을 재시작하지 마세요") 
        print("  3. 또는 수동으로 모델 경로를 지정하세요:")
        print("     best_model_path = '/content/runs/detect/your_model/weights/best.pt'")
        print("     그 후 이 셀을 다시 실행하세요")
    else:
        # 2️⃣ 모델 로드
        print(f"\n🤖 모델 로드 중...")
        test_model = YOLO(best_model_path)
        print(f"✅ 모델 로드 성공!")
        
        # 3️⃣ 2단계 데이터에서 테스트 이미지 직접 가져오기
        test_images = get_test_images_from_step2()
        
        if not test_images:
            print("❌ 테스트용 이미지를 찾을 수 없어 테스트를 중단합니다")
            print("💡 해결 방법:")
            print("  1. Google Drive에서 훈련 데이터가 올바르게 마운트되었는지 확인")
            print("  2. /content/drive/MyDrive/pinetree_scan/training_data/ 경로 확인")
            print("  3. TIF/TIFF 파일이 정상적으로 접근 가능한지 확인")
        else:
            # 4️⃣ 탐지 및 시각화 실행
            results_dir = save_dir or '/content/detection_test_results'
            os.makedirs(results_dir, exist_ok=True)
            
            detection_results, viz_paths = visualize_detection_results(test_model, test_images, results_dir)
            
            # 5️⃣ 성능 분석
            performance_stats = analyze_detection_performance(detection_results)
            
            # 6️⃣ 결과 백업 (Google Drive)
            if IN_COLAB and viz_paths:
                print(f"\n💾 Google Drive 결과 백업 중...")
                
                backup_dir = '/content/drive/MyDrive/pinetree_scan/detection_test_results'
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                final_backup_dir = f"{backup_dir}/detection_test_{timestamp}"
                
                os.makedirs(final_backup_dir, exist_ok=True)
                
                # 시각화 이미지 백업
                for viz_path in viz_paths:
                    if os.path.exists(viz_path):
                        backup_path = os.path.join(final_backup_dir, os.path.basename(viz_path))
                        shutil.copy2(viz_path, backup_path)
                
                print(f"✅ Google Drive 백업 완료!")
                print(f"  📁 백업 위치: MyDrive/pinetree_scan/detection_test_results/detection_test_{timestamp}")
                print(f"  📊 백업된 파일: {len(viz_paths)}개")
            
            # 7️⃣ 최종 요약
            print(f"\n🎉 성능 테스트 완료!")
            print(f"📊 테스트 요약:")
            print(f"  🤖 사용 모델: {os.path.basename(best_model_path)}")
            print(f"  📸 테스트 이미지: {len(test_images)}개")
            print(f"  🎯 총 탐지: {sum(r['detections'] for r in detection_results)}개")
            print(f"  📈 성능 등급: {performance_stats.get('performance_grade', 'N/A')}")
            
            # 전역 변수 업데이트
            globals()['detection_test_completed'] = True
            globals()['detection_results'] = detection_results
            globals()['test_performance'] = performance_stats
            
        # 메모리 정리
        if 'test_model' in locals():
            del test_model
            torch.cuda.empty_cache()

except Exception as e:
    print(f"❌ 성능 테스트 중 오류 발생: {str(e)[:100]}")
    print(f"💡 해결 방법:")
    print(f"  1. 2단계 학습이 완료되었는지 확인")
    print(f"  2. GPU 메모리 부족 시 런타임 재시작")
    print(f"  3. 데이터셋 경로 확인")
    
    import traceback
    traceback.print_exc()

finally:
    # 최종 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

print("=" * 60)
print("✅ 3단계 완료!")
print("🎯 수행된 작업:")
print("  ✓ 학습된 모델로 실제 피해목 탐지")
print("  ✓ 탐지 결과 시각화 (바운딩박스 마킹)")
print("  ✓ 탐지 성능 통계 분석")
print("  ✓ Google Drive 결과 백업")
print("🚀 이제 모델의 실제 성능을 확인할 수 있습니다!")